# Fetching Pre and Post synaptic Neuron Location

## Libraries

In [1]:
# ! pip install standard-transform
# ! pip install --upgrade caveclient

In [2]:
import numpy as np
import pandas as pd
from caveclient import CAVEclient
from microns_phase3 import nda
from standard_transform import minnie_transform_vx

Connecting microns@db.datajoint.com:3306


Could not create the ~log table
Could not access table `microns_phase3_nda`.`~log`


## Preparation

Function that transforms the voxel coorindate into a depth coordinate (function given to us by the authors of the Microns Project)

In [3]:
t_form_v = minnie_transform_vx()

### Querying the data from CAVEclient

In [4]:
client = CAVEclient()

In [5]:
# client.auth.save_token("64ea0271652fd86dc7fb6376cd9d52b9")

We will use the latest version of the minnie65 dataset, i.e. 661

In [6]:
client = CAVEclient('minnie65_public_v661')

In [7]:
client.materialize.get_tables()

['baylor_gnn_cell_type_fine_model_v2',
 'nucleus_alternative_points',
 'connectivity_groups_v507',
 'proofreading_status_public_release',
 'allen_column_mtypes_v1',
 'allen_v1_column_types_slanted_ref',
 'aibs_column_nonneuronal_ref',
 'nucleus_ref_neuron_svm',
 'aibs_soma_nuc_exc_mtype_preds_v117',
 'baylor_log_reg_cell_type_coarse_v1',
 'apl_functional_coreg_forward_v5',
 'nucleus_detection_v0',
 'aibs_soma_nuc_metamodel_preds_v117']

These are all the available tables, but we are interested only in the *apl_functional_coreg_forward_v5*, that is the one containing the information about the proofread neurons

In [34]:
temp = client.materialize.query_table('aibs_column_nonneuronal_ref')

In [35]:
temp.head()

,id_ref,created_ref,valid_ref,volume,pt_supervoxel_id,pt_root_id,id,created,valid,target_id,classification_system,cell_type,pt_position,bb_start_position,bb_end_position
0,269398,2020-09-28 22:41:16.766778+00:00,t,26.438205,88262953194999153,864691135203453161,1,2023-03-18 14:17:23.967558+00:00,t,269398,aibs_coarse_nonneuronal,pericyte,"[170832, 213904, 21702]","[nan, nan, nan]","[nan, nan, nan]"
1,301005,2020-09-28 22:41:57.863484+00:00,t,28.584673,91004241640527552,864691135382650586,2,2023-03-18 14:17:23.968298+00:00,t,301005,aibs_coarse_nonneuronal,pericyte,"[190864, 191024, 20463]","[nan, nan, nan]","[nan, nan, nan]"
2,262944,2020-09-28 22:40:54.872330+00:00,t,30.349230,89031649328771167,864691134947909116,3,2023-03-18 14:17:23.968970+00:00,t,262944,aibs_coarse_nonneuronal,pericyte,"[176224, 173952, 22095]","[nan, nan, nan]","[nan, nan, nan]"
3,267014,2020-09-28 22:42:01.613059+00:00,t,31.845253,88261303793472938,864691132771104287,4,2023-03-18 14:17:23.969646+00:00,t,267014,aibs_coarse_nonneuronal,pericyte,"[170752, 201632, 20808]","[nan, nan, nan]","[nan, nan, nan]"
4,256677,2020-09-28 22:40:42.643378+00:00,t,35.175465,89517770907257607,864691136436419998,5,2023-03-18 14:17:23.970312+00:00,t,256677,aibs_coarse_nonneuronal,pericyte,"[179776, 125648, 22141]","[nan, nan, nan]","[nan, nan, nan]"


In [9]:
df = client.materialize.query_table('apl_functional_coreg_forward_v5')

We start by setting as the index he ID associated to the root of the synapse

In [13]:
df = df.set_index('pt_root_id')

In [14]:
df.head()

,id_ref,created_ref,valid_ref,volume,pt_supervoxel_id,id,created,valid,target_id,session,scan_idx,unit_id,field,residual,score,pt_position,bb_start_position,bb_end_position
pt_root_id,,,,,,,,,,,,,,,,,,
864691135568539372,255686,2020-09-28 22:40:42.632533+00:00,t,297.846047,88954888800920543,67007,2023-04-06 15:05:50.491194+00:00,t,255686,9,6,1913,2,8.941486,4.206369,"[175760, 126480, 15504]","[nan, nan, nan]","[nan, nan, nan]"
864691135568539372,255686,2020-09-28 22:40:42.632533+00:00,t,297.846047,88954888800920543,65810,2023-04-06 15:05:49.502144+00:00,t,255686,9,4,7548,6,9.380808,-5.165012,"[175760, 126480, 15504]","[nan, nan, nan]","[nan, nan, nan]"
864691135568539372,255686,2020-09-28 22:40:42.632533+00:00,t,297.846047,88954888800920543,54982,2023-04-06 15:05:15.938520+00:00,t,255686,8,7,4778,4,3.763804,5.341737,"[175760, 126480, 15504]","[nan, nan, nan]","[nan, nan, nan]"
864691135568539372,255686,2020-09-28 22:40:42.632533+00:00,t,297.846047,88954888800920543,46412,2023-04-06 15:04:43.197444+00:00,t,255686,7,5,1946,2,4.071072,3.306988,"[175760, 126480, 15504]","[nan, nan, nan]","[nan, nan, nan]"
864691135568539372,255686,2020-09-28 22:40:42.632533+00:00,t,297.846047,88954888800920543,11823,2023-04-06 15:03:01.038836+00:00,t,255686,5,3,1994,2,5.419972,6.644126,"[175760, 126480, 15504]","[nan, nan, nan]","[nan, nan, nan]"


In [ ]:
df.shape

In [ ]:
prop = df.shape[0]//4

In [ ]:
unit_keys_14 = df.iloc[:prop, :][['session', 'scan_idx', 'unit_id']].to_dict(orient='records')
unit_keys_14

In [ ]:
area_membership = (nda.AreaMembership & unit_keys_14).fetch('brain_area')